In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
os.chdir(r'C:\Users\user\Desktop')

In [33]:
data1 = pd.read_csv("data_250418/000_Et_H_CO_n.csv", header = None)
data2 = pd.read_csv("data_250418/002_Et_H_CO_H.csv", header = None)
data3 = pd.read_csv("data_250418/008_Et_H_CO_L.csv", header = None)
data4 = pd.read_csv("data_250418/028_Et_H_CO_M.csv", header = None)


In [34]:
data1 = data1.drop(columns = range(3))
data1 = data1.values

data1.shape

(2970, 8)

In [35]:
data2 = data2.drop(columns = range(3))
data2 = data2.values

data2.shape

(2970, 8)

In [36]:
data3 = data3.drop(columns = range(3))
data3 = data3.values

data3.shape

(2970, 8)

In [37]:
data4 = data4.drop(columns = range(3))
data4 = data4.values

data4.shape

(2970, 8)

In [38]:
n = 100
err_sd = 30

In [47]:
data_arr1 = np.zeros([n, data1.shape[0], data1.shape[1]])
data_arr2 = np.zeros([n, data2.shape[0], data2.shape[1]])
data_arr3 = np.zeros([n, data3.shape[0], data3.shape[1]])
data_arr4 = np.zeros([n, data4.shape[0], data4.shape[1]])

In [69]:
for i in range(n):
    np.random.seed(i)
    data_arr1[i, :, :] = data1 + np.random.normal(0, err_sd, size = (data1.shape[0], data1.shape[1]))
    data_arr2[i, :, :] = data2 + np.random.normal(0, err_sd, size = (data2.shape[0], data2.shape[1]))
    data_arr3[i, :, :] = data3 + np.random.normal(0, err_sd, size = (data3.shape[0], data3.shape[1]))
    data_arr4[i, :, :] = data4 + np.random.normal(0, err_sd, size = (data4.shape[0], data4.shape[1]))

In [87]:
# data_original: 4개 가스의 모든 관측치를 담은 데이터 행렬
data_original = np.zeros([4*n,  data1.shape[0], data1.shape[1]])

data_original[:n, :, :] = data_arr1
data_original[n:2*n, :, :] = data_arr2
data_original[2*n:3*n, :, :] = data_arr3
data_original[3*n:4*n, :, :] = data_arr4

In [ ]:
# Wavelet PCA
def Wave_PCA_Class(data_original, n):

    n_samples = 4*n
    n_sensors = 8
    T_original = data1.shape[0]
    T = 4096 # 2의 거듭제곱으로 조정 (2^12)

    # 웨이블렛 계수 추출
    # 센서당 추출할 계수 개수 (임의로 설정, 필요 시 조정)
    """library(wavethresh)"""
    import pywt

    # 데이터 길이 조정 (T_original -> T, 뒤에 0 추가)
    data = np.zeros([n_samples, T, n_sensors])
    for i in range(n_samples):
        for j in range(n_sensors):
            data[i, :T_original, j] = data_original[i, :, j] # 원래 데이터
            data[i, (T_original + 1): T, j] = 0 # 뒤에 0으로 패딩

    # 학습 레이블 (4가지 가스, 각 100개 샘플)
    labels = np.repeat(np.arange(1, 5), n)

    # 1. 학습 데이터에 웨이블렛 계수 추출
    w_per_sensor = T_original
    wavelet_features = np.zeros([n_samples, n_sensors * w_per_sensor])

    for i in range(n_samples):
        for j in range(n_sensors):
            sensor_data = data[i, :, j]
            dwt_result = 